In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

import random
import os
import gc

import xgboost as xgb

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter('ignore')


In [ ]:
class CFG:
    path = "../input/playground-series-s5e6/"
    original_path = "../input/fertilizer-prediction/"

    n_splits = 10
    n_repeats = 10
    seed = 42
    
    learning_rate = 5e-2
    num_boost_round = 10000
    early_stopping_rounds = 100
    verbose_eval = 500

    target = "Fertilizer Name"

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()


In [ ]:
train = pd.read_csv(CFG.path + "train.csv").drop(columns=['id'])
test = pd.read_csv(CFG.path + "test.csv").drop(columns=['id'])

In [ ]:
original = pd.read_csv(CFG.original_path + "Fertilizer Prediction.csv")
original_copy = original.copy()
for i in range(5):
    original = pd.concat([original, original_copy], ignore_index=True)

In [ ]:
def add_catfeatures(df, cols):
    for col in cols:
        df[f"{col}_Binned"] = df[col].astype("category")
    return df

num_cols = [col for col in test.select_dtypes(exclude=['object', 'category']).columns]
train = add_catfeatures(train, num_cols)
test = add_catfeatures(test, num_cols)
original = add_catfeatures(original, num_cols)

In [ ]:
def add_catfeatures(df, cols):
    for col in cols:
        df[f"{col}_Binned"] = df[col].astype("category")
    return df

num_cols = [col for col in test.select_dtypes(exclude=['object', 'category']).columns]
train = add_catfeatures(train, num_cols)
test = add_catfeatures(test, num_cols)
original = add_catfeatures(original, num_cols)
def rename_temperature_column(df):
    df = df.rename(columns={'Temparature': 'Temperature'})
    return df
    
train = rename_temperature_column(train)
test = rename_temperature_column(test)
original = rename_temperature_column(original)

In [ ]:
cat_cols = [col for col in test.select_dtypes(include=['object', 'category']).columns]

for col in cat_cols:
    label_enc = LabelEncoder()
    train[col] = label_enc.fit_transform(train[col])
    test[col] = label_enc.transform(test[col])
    original[col] = label_enc.transform(original[col])

for col in cat_cols:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")
    original[col] = original[col].astype("category")

del label_enc
gc.collect()
target_label_enc = LabelEncoder()
train[CFG.target] = target_label_enc.fit_transform(train[CFG.target])
original[CFG.target] = target_label_enc.transform(original[CFG.target])
features = [col for col in test.columns if col != CFG.target]
for col in features:
    if train[col].nunique() == 1:
        features.remove(col)

In [ ]:
train[features]
original[features]
test[features]

In [ ]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [ ]:
# XGBoost parameters
params = {
    'objective': 'multi:softprob',
    'num_class': train[CFG.target].nunique(),
    'seed': CFG.seed,
    'max_depth': 8,
    'max_bins': 128,
    'learning_rate': CFG.learning_rate,
    'min_child_weight': 2,
    'alpha': 6.5,
    'reg_lambda': 5.3,
    'subsample': 0.8,
    'colsample_bytree': 0.3,
    'tree_method': 'hist',  
    'device': "cuda"
}

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Image

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")
original = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
original.info()

In [ ]:
# Add 'source' label
#original['Source'] = 'real'
#train['Source'] = 'synth'

# Drop the 'id' column
X = train.drop(columns=['id', 'Fertilizer Name'])

# Extract the target column
y = train['Fertilizer Name']

X.head()

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Output labels are now numbers
y_encoded[:10]

In [ ]:
# Just check
X.info()

In [ ]:
# Prepare multiple copies of original dataset
orig_copy = original.copy()

# Number of copies
n = 6
for i in range(n):
    original = pd.concat([original, orig_copy], axis=0, ignore_index=True)
    
original.info()

In [ ]:
# Identify underrepresented classes
train['Fertilizer Name'].value_counts()

In [ ]:
# Identify underrepresented classes
orig_copy['Fertilizer Name'].value_counts()

In [ ]:
# Store scores
f1_scores = []
map3_scores = []
models = []

# Collect predictions and true labels across all folds
all_y_true = []
all_y_pred = []

# Prepare K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded)):
    print(f"\n***** Fold {fold + 1} *****")

    # Make full copies to avoid warnings
    X_train = X.iloc[train_idx].copy()
    X_val = X.iloc[val_idx].copy()
    y_train = y_encoded[train_idx]
    y_val = y_encoded[val_idx]

    # Combine original with train data 
    X_train = pd.concat([X_train, original], ignore_index=True)
    y_train = np.concatenate([y_train, le.transform(original['Fertilizer Name'])])

    # Drop target column from training data
    X_train.drop(columns=['Fertilizer Name'], inplace=True)

    # Convert all features to categorical (except target, which is already separated)
    for col in X_train.columns:
        X_train[col] = X_train[col].astype('category')
        
    for col in X_val.columns:
        X_val[col] = X_val[col].astype('category')
    
    cat_features = X_train.columns.tolist()   # capture all input columns

    # For debugging purposes
    # print(cat_features)
    # print(X_train.info())
    # print(X_val.info())

    model = XGBClassifier(
                max_depth=7,
                colsample_bytree=0.4,
                subsample=0.8,
                n_estimators=10000,
                learning_rate=0.01,
                gamma=0.26,
                max_delta_step=4,
                reg_alpha=2.7,
                reg_lambda=1.4,
                objective='multi:softprob',
                random_state=13,
                enable_categorical=True,
                tree_method='hist',     
                device='cuda'  
            )

    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train),(X_val, y_val)],
        early_stopping_rounds=50,
        verbose=500,
    )
    
    # Predict class labels and probabilities
    y_pred = model.predict(X_val)
    y_probs = model.predict_proba(X_val)

    # Store predictions and true labels
    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)

    # F1 Score
    report = classification_report(y_val, y_pred, output_dict=True)
    f1_macro = report["macro avg"]["f1-score"]
    f1_scores.append(f1_macro)
    
    # MAP@3
    top3_preds = np.argsort(y_probs, axis=1)[:, -3:][:, ::-1]
    
    def mapk(actual, predicted, k=3):
        def apk(a, p, k):
            if a in p[:k]:
                return 1.0 / (p[:k].index(a) + 1)
            return 0.0
        return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

    map3 = mapk(y_val.tolist(), top3_preds.tolist(), k=3)
    map3_scores.append(map3)
    models.append(model)

    print(f"F1 (macro): {f1_macro:.4f} | MAP@3: {map3:.4f}")

# Final Results
print("\n***** Final CV Results *****")
print(f"Avg F1: {np.mean(f1_scores):.4f}")
print(f"Avg MAP@3: {np.mean(map3_scores):.4f}")

In [ ]:
# Loss curves for the last fold
results = model.evals_result()
plt.plot(results['validation_0']['mlogloss'], label='Train')
plt.plot(results['validation_1']['mlogloss'], label='Val')
plt.legend()
plt.show()

In [ ]:
# Step 1: Confusion matrix
cm = confusion_matrix(all_y_true, all_y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Step 2: Make it pretty
plt.figure(figsize=(8, 6))
sns.heatmap(cm_norm, annot=True, fmt=".3f", cmap="Greens", 
            xticklabels=le.classes_, yticklabels=le.classes_,
           )

plt.title("Normalized Confusion Matrix", fontsize=16)
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred, digits=4))

In [ ]:
# Initialize accumulator
importances_total = np.zeros(len(cat_features))
feature_names = cat_features

# Accumulate importance per model
for model in models:
    importances_total += model.feature_importances_

# Average
importances_avg = importances_total / len(models)

# Make a dataframe
importances_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances_avg
}).sort_values(by='importance', ascending=False)
# Limit to top N features for readability, default to all columns
top_n = len(X_train.columns)
top_features = importances_df.head(top_n)

# Create a green color palette
green_palette = sns.color_palette("Greens", as_cmap=False, n_colors=len(top_features))

# Plot using barplot
plt.figure(figsize=(8, 5))

sns.barplot(
    data=top_features,
    y='feature',
    x='importance',
    palette=green_palette
)

plt.title("Top Feature Importances")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
# y_true should be integer labels (encoded)
# y_proba is shape (n_samples, n_classes)
n_classes = y_probs.shape[1]

# Initialize matrix
soft_cm = np.zeros((n_classes, n_classes))

# Fill it
for i in range(len(y_val)):
    true_class = y_val[i]
    soft_cm[true_class] += y_probs[i]  # adds vector of predicted probs

# Normalize each row (optional)
soft_cm_normalized = soft_cm / soft_cm.sum(axis=1, keepdims=True)

# Plot it!
plt.figure(figsize=(8, 6))
sns.heatmap(soft_cm_normalized, annot=True, fmt=".3f", cmap="Greens",
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted Class", fontsize=12)
plt.ylabel("True Class", fontsize=12)
plt.title("Soft Confusion Matrix (Probabilities)", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
def top_n_coverage(y_true, y_proba, n=3):
    """Returns the proportion of times the true label is in the top-N predicted labels."""
    top_n_preds = np.argsort(y_proba, axis=1)[:, -n:]  # Get top N indices (classes)
    
    # Check if the true label is in the top N predictions
    hits = [y_true[i] in top_n_preds[i] for i in range(len(y_true))]
    
    return np.mean(hits)

for n in range(1, 8):
    coverage = top_n_coverage(y_val, y_probs, n)
    print(f"Top-{n} Coverage: {coverage:.4f}")

In [ ]:
# Setup: Top-N values
top_ns = list(range(1, y_probs.shape[1] + 1))
coverages = [top_n_coverage(y_val, y_probs, n) for n in top_ns]

# Apply Seaborn theme
sns.set(style="whitegrid")
sns.set_palette("Greens")

# Create figure
plt.figure(figsize=(8, 5))

# Plot with seaborn line aesthetics
sns.lineplot(x=top_ns, y=coverages, marker='o', color='green', linewidth=2)

# Decorations
plt.title("Top-N Coverage Curve", fontsize=16)
plt.xlabel("N (Top-N Predictions)", fontsize=12)
plt.ylabel("Coverage", fontsize=12)
plt.ylim(0, 1.05)
plt.xticks(top_ns)
plt.yticks([i/10 for i in range(11)])
plt.grid(True, linestyle='--', alpha=0.6)
plt.axvline(x=3, color='red', linestyle='--', linewidth=2, label='Top-3 Threshold')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Convert test data
for col in cat_features:
    test[col] = test[col].astype('category')

# Accumulate prediction probabilities
all_preds = np.zeros((test.shape[0], len(le.classes_)))

X_test = test.drop(columns='id')
cat_features = X_test.columns.tolist()   # capture all input columns

for model in models:
    probs = model.predict_proba(X_test)
    all_preds += probs

# Average over folds
avg_preds = all_preds / len(models)

# Get top 3 indices like before
top3_preds = np.argsort(probs, axis=1)[:, -3:][:, ::-1]  # Top 3 class indices, descending order

# Convert class indices back to original label strings
top3_labels = le.inverse_transform(top3_preds.ravel()).reshape(top3_preds.shape)

submission = pd.DataFrame({
    'id': test['id'],  # Replace with actual ID column name
    'Fertilizer Name': [' '.join(row) for row in top3_labels]
})

submission.to_csv('submission.csv', index=False)
print("Done!")

In [ ]:
submission.head()